# Retrieval-Augmented Generation (RAG)

In [1]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

In [7]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook
chroma_client = chromadb.PersistentClient("../chroma")
nuntrition_qna_db = chroma_client.get_collection(name="nutrition_qna")

In [8]:
results = nuntrition_qna_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

[('is_pregnancy', False)]
Question: What is the recommended amount of bananas I should consume to count it as a single serving?
        Answer: One small-sized banana can be counted as a single serving.

        This Q&A pair provides information about nutrition and health topics.


[('is_pregnancy', False)]
Question: Which food is recommended for infants after they've been introduced to ripe bananas and sweet potatoes?
        Answer: Introduce porridge made from wheat flour or ground rice, starting with only one cereal. Once a week has passed, you may increase the frequency of this new food to two feedings per day.

        This Q&A pair provides information about nutrition and health topics.




In [10]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too look up calorie information.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = calories_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

@function_tool
def nutrtition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too ask a question about nutrition.

    Args:
        query: The question to ask
        max_results: The maximum number of results to return.

    Returns:
        A string containing the question and the answer related to the query.
    """

    results = nuntrition_qna_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Related answers to your question:\n" + "\n".join(formatted_results)

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [11]:
calorie_lookup_tool('bananas')

TypeError: 'FunctionTool' object is not callable

In [12]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information and nutrtion advice    .
    You give concise answers.
    
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrtition_qna_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[calorie_lookup_tool, nutrtition_qna_tool],
)

In [14]:
with trace("Nutrition Assistant with Nutrition and Calorie RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are the best meal choices for pregnant women and how many calories do they have?",
    )
    print(result.final_output)

Key points:
- Focus on nutrient-dense meals: iron-rich foods, folate, calcium, protein, healthy fats (DHA), and fiber.
- Calorie guidance: add about 200 extra calories per day (one additional small meal) during pregnancy.

Sample meal ideas:
- Breakfast: fortified cereal with milk, plus a small fruit
- Lunch: spinach salad with chickpeas, tomatoes, feta, and vinaigrette; whole-grain bread
- Snack: Greek yogurt with berries
- Dinner: grilled chicken or beans with quinoa, steamed broccoli, and a slice of avocado

Note: Avoid unpasteurized dairy, raw or undercooked eggs/meat, high-mercury fish, and limit processed junk foods. Talk to your healthcare provider for personalized needs.
